In [1]:
import numpy as np

#### Multiclass classification case

In [18]:
numExperts = 5
numClasses = 4
numData = 500
biases = [0.01, 0.2, 0.4, 0.05, 0.25]

In [19]:
def genConfMat(numExperts, numClasses, biases):
    """
    INPUT-
    numExperts - number of Experts
    numClasses - number of Classes
    biases - biases for each expert - Chance of misclassification to be encoded in Conf Matrix
    
    OUTPUT-
    List of confusion matrices
    """
    confMatList = []
    #Mandatory check before generating the data
    assert len(biases) == numExperts
    #Iterating over each expert to create a confusion matrix
    for exp in range(numExperts):
        confMat = np.eye(numClasses, dtype=float)
        confMat = confMat + biases[exp]*np.ones((numClasses, numClasses), dtype=float)
        rowSum = np.sum(confMat, axis=1)
        #normalizing the confusion matrix
        confMatNorm = confMat/rowSum[:,np.newaxis]
        confMatList.append(confMatNorm)
    return confMatList

In [20]:
def genTrueClasses(numData, numClasses, classDist='uniform'):
    """
    INPUT
    numData - number of data points to be generated
    numClasses - number of classes
    classDist - type of distribution governing true classes in the data
    
    OUTPUT
    data - one hot vectorized data
    data_num - data with the classes as integers
    """
    if classDist=='uniform':
        data_num = np.random.choice(range(numClasses), numData)
        data = np.eye(numClasses)[data_num]
        return data, data_num

In [21]:
def genExpertLabels_MC(trueClsNum, confMatList, numExperts, numClasses):
    """
    INPUT
    trueClsNum - data with the classes as integers
    confMatList - List of confusion matrices
    numExperts - number of Experts
    numClasses - number of classes
    
    OUTPUT
    Data in a 3d array format
    """
    #Mandatory check before generating expert labels
    assert len(confMatList) == numExperts
    numData = len(trueClsNum)
    dataList = []
    for data_idx in range(numData):
        true_class = trueClsNum[data_idx]
        expert_val = np.zeros((numExperts, numClasses))
        for exp in range(numExperts):
            confMat = confMatList[exp]
            predClass = np.random.choice(range(numClasses), p=confMat[true_class,:])
            expert_val[exp, predClass] = 1
        dataList.append(expert_val)
    return np.array(dataList)

In [22]:
confMatList = genConfMat(numExperts, numClasses, biases)

In [23]:
trueClasses, trueClsNum = genTrueClasses(numData, numClasses)

In [24]:
expertData = genExpertLabels_MC(trueClsNum, confMatList, numExperts, numClasses)

In [25]:
expertData.shape

(500, 5, 4)

In [26]:
np.save('MC_500_5_4.npz',expertData)

In [27]:
np.save('MC_500_5_4_reference_classes', trueClsNum)

In [30]:
for i in range(numExperts):
    print "Confusion Matrix for expert ",str(i+1)
    print confMatList[i]

 Confusion Matrix for expert  1
[[ 0.97115385  0.00961538  0.00961538  0.00961538]
 [ 0.00961538  0.97115385  0.00961538  0.00961538]
 [ 0.00961538  0.00961538  0.97115385  0.00961538]
 [ 0.00961538  0.00961538  0.00961538  0.97115385]]
Confusion Matrix for expert  2
[[ 0.66666667  0.11111111  0.11111111  0.11111111]
 [ 0.11111111  0.66666667  0.11111111  0.11111111]
 [ 0.11111111  0.11111111  0.66666667  0.11111111]
 [ 0.11111111  0.11111111  0.11111111  0.66666667]]
Confusion Matrix for expert  3
[[ 0.53846154  0.15384615  0.15384615  0.15384615]
 [ 0.15384615  0.53846154  0.15384615  0.15384615]
 [ 0.15384615  0.15384615  0.53846154  0.15384615]
 [ 0.15384615  0.15384615  0.15384615  0.53846154]]
Confusion Matrix for expert  4
[[ 0.875       0.04166667  0.04166667  0.04166667]
 [ 0.04166667  0.875       0.04166667  0.04166667]
 [ 0.04166667  0.04166667  0.875       0.04166667]
 [ 0.04166667  0.04166667  0.04166667  0.875     ]]
Confusion Matrix for expert  5
[[ 0.625  0.125  0.125  

#### Yes/No Question data generation

The assumption is that all the questions are asked in the same order. This is done to maintain uniformity while generating the data.

The first column indicates NO, whereas the second column indicates YES.

In [79]:
def genExpertLabels_YN(trueClsNum, confMatList, numExperts, numClasses):
    """
    INPUT
    trueClsNum - data with the classes as integers
    confMatList - List of confusion matrices
    numExperts - number of Experts
    numClasses - number of classes
    
    OUTPUT
    Data in a 4d array format
    """
    #Mandatory check before generating expert labels
    assert len(confMatList) == numExperts
    numData = len(trueClsNum)
    dataList = []
    for data_idx in range(numData):
        true_class = trueClsNum[data_idx]
        expert_val = np.zeros((numExperts, numClasses, 2))
        for exp in range(numExperts):
            confMat = confMatList[exp]
            for qClass in range(numClasses):
                prob = confMat[true_class, qClass]
                val = np.random.binomial(1, prob)
                expert_val[exp, qClass, val] = 1
        dataList.append(expert_val)
    return np.array(dataList)

In [80]:
expertData_YN = genExpertLabels_YN(trueClsNum, confMatList, numExperts, numClasses)

In [81]:
expertData_YN.shape

(10, 5, 4, 2)

#### Comparing two classes and asking the expert to pick either

The number of questions for every datapoint if we have $k$ classes will be $\frac{k(k-1)}{2}$

In [82]:
import itertools

In [83]:
order = []
for subset in itertools.combinations(range(numClasses), 2):
    order.append(subset)

Make sure to save order as well, just in case we might need it in the inference process.

In [89]:
def genExpertLabels_CC(trueClsNum, confMatList, numExperts, numClasses, order):
    """
    INPUT
    trueClsNum - data with the classes as integers
    confMatList - List of confusion matrices
    numExperts - number of Experts
    numClasses - number of classes
    order - List with tuples of classes being compared (All combinations)
    OUTPUT
    Data in a 4d array format
    """
    #Mandatory check before generating expert labels
    assert len(confMatList) == numExperts
    numData = len(trueClsNum)
    dataList = []
    for data_idx in range(numData):
        true_class = trueClsNum[data_idx]
        expert_val = np.zeros((numExperts, len(order), 2))
        for exp in range(numExperts):
            confMat = confMatList[exp]
            for qClass_idx in range(len(order)):
                qClass = order[qClass_idx]
                prob_neg = confMat[true_class, qClass[0]]
                prob_pos = confMat[true_class, qClass[1]]
                prob = prob_pos/(prob_pos + prob_neg)
                val = np.random.binomial(1, prob)
                expert_val[exp, qClass_idx, val] = 1
        dataList.append(expert_val)
    return np.array(dataList)

In [108]:
expertData_CC = genExpertLabels_CC(trueClsNum, confMatList, numExperts, numClasses, order)

In [113]:
expertData_CC.shape

(10, 5, 6, 2)